In [20]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [21]:
using LightGBM
using RDatasets
using DataStructures
using MLBase
using Random

In [22]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [23]:
iris = dataset("datasets", "iris")
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

51-element Vector{Int64}:
   1
   2
   5
   7
  10
  11
  12
  25
  28
  31
   ⋮
 124
 130
 133
 134
 135
 138
 145
 146
 150

In [24]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))


assign_class (generic function with 1 method)

In [25]:
estimator = LGBMClassification(
    objective = "binary",
    num_iterations = 100,
    learning_rate = .1,
    early_stopping_round = 5,
    feature_fraction = .8,
    bagging_fraction = .9,
    bagging_freq = 1,
    num_leaves = 1000,
    num_class = 1,
    metric = ["auc", "binary_logloss"]
)

LGBMClassification(LightGBM.Booster(Ptr{Nothing} @0x0000000000000000, LightGBM.Dataset[]), "", "binary", "gbdt", 100, 0.1, 1000, -1, "serial", 12, -1.0, 100, 0.001, 0.0, 0.0, 0.0, 0.0, 0.8, 1.0, 2, 0.9, 1.0, 1.0, 1, 3, 5, false, 6, 255, 200000, 1, "", true, false, Int64[], true, true, false, true, 1.0, 1.0, 0.1, 50, 0.5, false, false, 4, 0.2, 0.1, 100, 32, 10.0, 10.0, ["auc", "binary_logloss"], 1, false, Int64[], 1, 12400, 120, "", 1, "cpu", false, -1, -1, 1, false, false)

In [38]:
fit!(estimator, X[trainids,:], y[trainids], (X[testids,:], y[testids]))

MethodError: MethodError: no method matching fit!(::LGBMClassification, ::Matrix{Float64}, ::Vector{Int64}, ::Tuple{Matrix{Float64}, Vector{Int64}})
Closest candidates are:
  fit!(!Matched::StatisticalModel, ::Any...) at C:\Users\tsakalos\.julia\packages\StatsBase\IPydo\src\statmodels.jl:183

In [37]:
q = X[testids,:];
predictions_RF = LightGBM.predict(estimator, q)
findaccuracy(predictions_RF,y[testids])

ErrorException: Estimator does not contain a fitted model.

In [28]:
pred = predict(bst, X[testids,:])
print("test-error=", sum((pred .> 0.5) .!= y[testids]) / float(size(pred)[1]), "\n")

UndefVarError: UndefVarError: predict not defined